# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

/opt/anaconda3/envs/langchain_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [5]:
import os
from dotenv import load_dotenv

load_dotenv(".env")
load_dotenv(".env copy")
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [6]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [8]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [9]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_97172/3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [10]:
OBJECTIVE = "Write a weather report for SF today"

In [11]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [12]:
baby_agi({"objective": OBJECTIVE})

/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_97172/3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any potential weather events.

2. Check reliable sources: Make sure to check reliable sources such as the National Weather Service or local news stations for accurate and up-to-date information.

3. Determine the format: Decide on the format for the weather report. Will it be a written report, a video, or a combination of both? This will help guide the rest of the process.

4. Create an outline: Before diving into writing, create an outline for the weather report. This will help organize the information 

{'objective': 'Write a weather report for SF today'}

In [81]:
OBJECTIVE = "Create a short 3-day digital detox plan for office workers."

In [82]:
llm = OpenAI(temperature=0, max_tokens=320)

In [83]:
todo_prompt = PromptTemplate.from_template(
    "Make a concise 3-day digital detox plan for: {objective}. "
    "Include: daily goal, 2–3 offline activities, and 1 reflection prompt per day."
)
todo_chain = LLMChain(llm=llm, prompt=todo_prompt)

def plan_todo(input_text: str = "") -> str:
    return todo_chain.run({"objective": (input_text or OBJECTIVE)})

tools = [
    Tool(
        name="Search",
        func=SerpAPIWrapper(params={"engine": "google", "hl": "en", "gl": "de"}).run,
        description=(
            "Use this FIRST to find 1–2 credible sources about digital detox or workplace well-being. "
            "Return short bullet summaries with the URLs."
        ),
    ),
    Tool(
        name="TODO",
        func=plan_todo,
        return_direct=True,  # final answer comes straight from here
        description="Make a concise 3-day digital detox plan for: {objective}.\n"
    "Include: daily goal, 2–3 offline activities, and 1 reflection prompt per day.\n"
    "If the input contains lines starting with 'http', append a 'Sources:' section listing those URLs."
    ),
]

prefix = (
    "Objective: {objective}\n"
    "Context: {context}\n"
    "Instructions:\n"
    "- First, call the Search tool to find 1–2 credible sources (return short bullets with their URLs).\n"
    "- Then, call the TODO tool. Pass the objective AND the found URLs in the input.\n"
    "- The final answer must be ONLY the 3-day plan (no thoughts), and include a 'Sources:' section with the URLs."
)
suffix = "Task: {task}\n{agent_scratchpad}"

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [84]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [t.name for t in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True,)

# reset vectorstore
import faiss
from langchain.docstore import InMemoryDocstore

embedding_size = 1536  # OpenAIEmbeddings dim
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,          
    task_execution_chain=agent_executor,
    verbose=False,
    max_iterations=2,
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [85]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...

Question: Create a short 3-day digital detox plan for office workers.
Thought: A digital detox can help improve workplace well-being and productivity.
Action: Search
Action Input: "digital detox" OR "workplace well-being"
Observation: ['The study concludes that digital detox practices can significantly promote eudaimonic well-being by providing cognitive and emotional benefits.', 'Research has documented increased stress levels, decreased well-being, performance and challenges related to work-life balance (de Wet & Koekemoer, 2016; Jena, ...', 'Discover how digital detoxes improve mental health and productivity, and how tools like CoreHealth can make unplugging a workplace priority.', 'This paper aimed to address the issues in achieving employee wellbeing at the digital workplace and provide a viable solution to ensure digital wellbeing.', 'The excessive use of s

{'objective': 'Create a short 3-day digital detox plan for office workers.'}